In [2]:
import numpy as np
# import cupy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.utils.data as Data
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import scipy.io
import csv
import os
from VAE import myVAE, myVAE2d, vaeTest, vaeGenerate, vaeGenerateMultiSigma, vaeGenerateSigma, myMemPolyVAE
from classifier_cnn import Classifier, Classifier2D, addNoise2Batch
from GAN_SDR import ClassifierSDR, Discriminator

/Users/juntingdeng/anaconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class ClassifierPABinary(nn.Module):
    def __init__(self,nSDR):
        super().__init__()

        #For 800 length 
        self.conv0 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=5,stride=1, padding=2) # => 8*800
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5,stride=1, padding=2) # => 8*800
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=8,stride=2, padding=3) # => 16*400
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=6, stride=2, padding=2) # => 16*200

        self.linear1 = nn.Linear(in_features=32*200, out_features=256)
        self.linear2 = nn.Linear(in_features=256, out_features=nSDR)

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):

        #For 800 length 
        x=F.relu(self.conv0(x))
        x=F.relu(self.conv1(x))
        x=F.relu(self.conv2(x))
        x=F.relu(self.conv3(x))

        # x=self.maxpool1(x)
        flatten = nn.Flatten()
        x=flatten(x)
        x=F.relu(self.linear1(x))
        x=self.linear2(x)
        # x=F.sigmoid(x)

        return x

In [7]:
def train(nSDR, net, n_epochs, lr, batchsize,trainLoader, valLoader, testLoader,device, snr, noise):
    n_epochs=n_epochs
    optimizer = optim.Adam(net.parameters(), lr=lr)
    trainLoss_curve=[]
    trainAcc_curve=[]
    trainAccDict_curve=[]
    valLoss_curve=[]
    valAcc_curve=[]
    valAccDict_curve=[]
    for epoch in range(n_epochs):
        # print("epoch: ", epoch)
        net.train()
        train_loss = 0
        train_acc = 0
        train_acc_dict = torch.zeros((nSDR,2)).to(device)
        val_loss = 0
        val_acc=0
        val_acc_dict = torch.zeros((nSDR,2)).to(device)
        loss = None
        for batch_idx, (data, labels) in enumerate(trainLoader):
            data, labels = data.to(device, dtype=torch.float), labels.to(device,  dtype=torch.int64)
            optimizer.zero_grad()
            labels_ = net.forward(data)
            loss = F.cross_entropy(labels_, labels)
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
            labels_ = torch.argmax(labels_, dim=1)
            train_acc+=(sum(labels_==labels)/batchsize)
            for i in range(batchsize):
                label=labels[i]
                train_acc_dict[label][0]+=1
                if labels_[i]==label:
                    train_acc_dict[label][1]+=1
            

        for batch_idx, (data, labels) in enumerate(valLoader):
            data, labels = data.to(device, dtype=torch.float), labels.to(device,  dtype=torch.int64)
            optimizer.zero_grad()
            labels_ = net.forward(data)
            loss = F.cross_entropy(labels_, labels)
            val_loss+=loss.item()
            labels_ = torch.argmax(labels_, dim=1)
            val_acc+=(sum(labels_==labels)/batchsize)
            for i in range(batchsize):
                label=labels[i]
                val_acc_dict[label][0]+=1
                if labels_[i]==label:
                    val_acc_dict[label][1]+=1

        train_loss = train_loss/len(trainLoader)
        trainLoss_curve.append(train_loss)
        train_acc = train_acc/len(trainLoader)
        trainAcc_curve.append(train_acc.item())

        val_loss = val_loss/len(valLoader)
        valLoss_curve.append(val_loss)
        val_acc = val_acc/len(valLoader)
        valAcc_curve.append(val_acc.item())
        
        train_acc_spt = torch.round(train_acc_dict[:,1]/train_acc_dict[:,0], decimals=4)
        val_acc_spt = torch.round(val_acc_dict[:,1]/val_acc_dict[:,0], decimals=4)
        if (epoch+1)%10==0:
            print('\nEpoch: {} \tTraining Loss: {:.6f}, Training Acc: {:.6f}'.format(epoch, train_loss, train_acc))
            print('Epoch: {} \tValidation Loss: {:.6f}, Validation Acc: {:.6f}'.format(epoch, val_loss, val_acc))
            print("Train acc_dic \t{}".format(train_acc_spt.detach().cpu().numpy()))
            print("Val acc_dic \t{}".format(val_acc_spt.detach().cpu().numpy()))
            checkpoint_path="../../../RFF-RL/Code/checkpoint/Classifier/binaryPA_epoch"+str(epoch+1)+".cnn"
            torch.save({
                "epoch": epoch,
                'lr': lr,
                'batch_size': batchsize,
                'optimizer': optimizer,
                'classifier': net,
                'train_acc': train_acc,
                'val_acc': val_acc,
            }, checkpoint_path)

    # test
    test_acc=0
    test_acc_dict = torch.zeros((nSDR,2)).to(device)
    for batch_idx, (data, labels) in enumerate(testLoader):
        data, labels = data.to(device, dtype=torch.float), labels.to(device,  dtype=torch.int64)
        optimizer.zero_grad()
        labels_ = net.forward(data)
        labels_ = torch.argmax(labels_, dim=1)
        test_acc+=(sum(labels_==labels)/batchsize)
        for i in range(batchsize):
            label=labels[i]
            test_acc_dict[label][0]+=1
            if labels_[i]==label:
                test_acc_dict[label][1]+=1

    test_acc = test_acc/len(testLoader)

    test_acc_spt = torch.round(test_acc_dict[:,1]/test_acc_dict[:,0], decimals=4)
    print('Test Acc: {:.6f}'.format(test_acc))
    print("Test acc_dic \t{}".format(test_acc_spt.detach().cpu().numpy()))

    # if save model
    if not noise:
        checkpoint_path="../../../RFF-RL/Code/checkpoint/Classifier/11SDR_nonoise_binaryPA_epoch"+str(n_epochs)+".cnn"
    else:
        checkpoint_path="../../../RFF-RL/Code/checkpoint/Classifier/11SDR_binaryPA_epoch"+str(n_epochs)+".cnn"
    torch.save({
        "epoch": epoch,
        'lr': lr,
        'batch_size': batchsize,
        'optimizer': optimizer,
        'classifier': net,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc 
    }, checkpoint_path)
    

    train_acc_spt = train_acc_spt.detach().cpu().numpy()
    val_acc_spt = val_acc_spt.detach().cpu().numpy()
    test_acc_spt = test_acc_spt.detach().cpu().numpy()

    plt.figure()
    plt.plot(trainLoss_curve, label="Train Loss")
    plt.plot(valLoss_curve, label="Val Loss")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Loss")

    plt.figure()
    plt.plot(trainAcc_curve, label="Train")
    plt.plot(valAcc_curve, label="Val")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Epochs")
    plt.show()

    plt.figure()
    plt.plot(train_acc_spt, label="Train Acc {:.2f}%".format(train_acc*100))
    plt.plot(val_acc_spt, label="Validation Acc {:.2f}%".format(val_acc*100))
    plt.plot(test_acc_spt, label="Test Acc {:.2f}%".format(test_acc*100))
    plt.legend()
    plt.xlabel("RFF")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs RFFs")
    plt.show()


In [4]:
## train binary classifier at 0.25 meters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device: ", device)
PATH="../../../RFF-RL/TcasRFFData/"
dataFile="ChipCAIRprocessedType3BlockOS20len800comb220V1p2Quant16.npz"
file = np.load(PATH+dataFile)
items = file.files
items = file.files
datasets_all, targets_all = file[items[0]], file[items[1]]
size=300
datasets=np.zeros((220*size, 2, 800))
targets=np.ones((220*size,))
for i in range(220):
    datasets[i*size:(i+1)*size] = datasets_all[i*730:i*730+size]


noise = False
ray=True
snr = 35
print("noise =", noise)
if noise:
    print("snr = ", snr)
    datasets_noised = addNoise2Batch(datasets, snr, device) #np array
    datasets_noised = (datasets_noised - np.min(datasets_noised))/(np.max(datasets_noised) - np.min(datasets_noised)) # ~[0,1]
    datasets_noised = 2*datasets_noised-1
    print(np.max(datasets_noised))
    print(np.min(datasets_noised))
    # datasets=np.expand_dims(datasets, axis=1) # add one dimension for harmonics at channel 1
    datasets_total = datasets_noised
    targets_total = targets
    
elif ray:
    h = np.random.rayleigh(1,(datasets.shape[0],1,800))
    datasets_ray=h*datasets
    datasets_ray = (datasets_ray - np.min(datasets_ray))/(np.max(datasets_ray) - np.min(datasets_ray)) # ~[0,1]
    datasets_ray = 2*datasets_ray-1
    datasets_total = np.concatenate((datasets,datasets_ray), axis=0)
    targets_total = np.concatenate((np.ones(datasets.shape[0]),np.zeros(datasets.shape[0])), axis=0)
    
else:
    datasets_total, targets_total = datasets, targets
print("datasets_total shape: ", datasets_total.shape)
print("targets_total: ", targets_total.shape)
np.random.seed(0)
datasets, targets = torch.tensor(datasets_total), torch.tensor(targets_total)
print("datasets range: ", datasets.max(), datasets.min())
print("targets range: ", targets.max(), targets.min())

device:  cuda
noise = False
datasets_total shape:  (132000, 2, 800)
targets_total:  (132000,)
datasets range:  tensor(1., dtype=torch.float64) tensor(-1., dtype=torch.float64)
targets range:  tensor(1., dtype=torch.float64) tensor(0., dtype=torch.float64)


In [12]:


np.random.seed(0)
datasets, targets = torch.tensor(datasets_total), torch.tensor(targets_total)
# targets = targets.type(torch.DoubleTensor)
SDRdatasets=Data.TensorDataset(datasets, targets)
train_set, val_set, test_set = Data.random_split(SDRdatasets, [0.6, 0.2, 0.2])

batch_size = 32
n_epochs = 2000
lr= 1e-3   #1e-3
bitarget=2
trainLoader = Data.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
valLoader = Data.DataLoader(val_set, batch_size=batch_size, shuffle=True, drop_last=True)
testLoader = Data.DataLoader(test_set, batch_size=batch_size, shuffle=True, drop_last=True)
# testLoader = Data.DataLoader(rffTest, batch_size=batch_size, shuffle=True, drop_last=True)
n_harmonics=datasets.shape[1]

net = ClassifierPABinary(nSDR=bitarget).to(device)
train(net=net, nSDR=bitarget, n_epochs=n_epochs, lr=lr, batchsize=batch_size,trainLoader=trainLoader, valLoader=valLoader, testLoader=testLoader,device=device, snr = snr, noise=noise)


Epoch: 9 	Training Loss: 0.000000, Training Acc: 1.000000
Epoch: 9 	Validation Loss: 0.000000, Validation Acc: 1.000000
Train acc_dic 	[1. 1.]
Val acc_dic 	[1. 1.]

Epoch: 19 	Training Loss: 0.000000, Training Acc: 1.000000
Epoch: 19 	Validation Loss: 0.000000, Validation Acc: 1.000000
Train acc_dic 	[1. 1.]
Val acc_dic 	[1. 1.]


KeyboardInterrupt: 

In [16]:
## test fake signal one time all on binary cnn
device='cuda'
nSDR=2
samples_perSDR = 500
classifier = ClassifierPABinary(nSDR=nSDR)
clf_load=torch.load("../../../RFF-RL/Code/checkpoint/Classifier/binaryPA_epoch20.cnn", map_location=device)
# clf_load=torch.load("./checkpoint/ClassifierSDR/11SDR_normseperate_epoch2000.cnn", map_location=device) 
classifier=clf_load["classifier"]

# datasets_total_fake = datasets_total[:][:][:]
# targets_total_fake = targets_total[:]
datasets_total_fake = np.zeros((6500, 2, 800))
targets_total_fake = np.zeros((6500, ))

path = "./checkpoint/gan/gan_pa_all/snr35/"
ckps = os.listdir(path)
batchsize=128
idx=-1
for ckp in ckps:
    dataset_fake = torch.zeros((samples_perSDR, 2, 800))
    datasets_total_fake=torch.tensor(datasets_total_fake, dtype=float).to(device)
    targets_total_fake=torch.tensor(targets_total_fake, dtype=torch.int64).to(device)

    if len(ckp.split("_"))<2:
        continue
    idx+=1
    RFFlabel = int(ckp.split("_")[0][3:])
    SNR = int(ckp.split("_")[1][3:])
    train_acc = float(ckp.split("_")[3][8:])
    val_acc = float(ckp.split("_")[4][6:])
    print("\nidx:{}, RFF:{}, snr:{}, tran_acc:{}, val_acc:{}".format(idx, RFFlabel, SNR, train_acc, val_acc))
    ckp_load=torch.load(path+ckp, map_location=device)
    generator = ckp_load["generator"]
    discriminator = ckp_load["discriminator"]
    classifier_load = ckp_load["classifier"]

    # data = data[:samples_perSDR]
    data=datasets_all[RFFlabel*730:RFFlabel*730+samples_perSDR]
    print("data shape:", data.shape)
    for sigIdx in range(data.shape[0]):
        data[sigIdx]=-1+2*((data[sigIdx] - data[sigIdx].min())/(data[sigIdx].max() - data[sigIdx].min()))
    data = addNoise2Batch(data, snr=SNR, device=device)
    data = torch.tensor(data, dtype=torch.float, device=device)
    
    realdata_loader = Data.DataLoader(data, batch_size=batchsize, shuffle=True, drop_last=True)
    for batch_idx, (data_batch) in enumerate(realdata_loader): 
        dataset_fake[batch_idx*batchsize:(batch_idx+1)*batchsize],mu, logvar = generator.forward(data_batch)
    if (batch_idx+1)*batchsize<samples_perSDR:
        repeat = samples_perSDR-(batch_idx+1)*batchsize
        dataset_fake[-repeat:]=dataset_fake[:repeat]

    datasets_total_fake[idx*samples_perSDR:(idx+1)*samples_perSDR]=dataset_fake
    # targets_total_fake[SDRlabel*samples_perSDR:(SDRlabel+1)*samples_perSDR]=np.zeros(samples_perSDR)

print("datasets: ", datasets_total_fake.shape)
print("targets: ", targets_total_fake.shape)
print("datasets range: ", datasets_total_fake.max(), datasets_total_fake.min())
print("targets range: ", targets_total_fake.max(), targets_total_fake.min())
# targets_total_fake = targets_total_fake.reshape((-1,1))

testdatasets_fake=Data.TensorDataset(datasets_total_fake, targets_total_fake)
testLoader_fake = Data.DataLoader(testdatasets_fake, batch_size=batchsize, shuffle=True, drop_last=True)

test_acc=0
fpr=0
test_acc_dict = torch.zeros((nSDR,2)).to(device)
test_acc_matrix = torch.zeros((nSDR, nSDR))
for batch_idx, (data, labels) in enumerate(testLoader_fake):
    data, labels = data.to(device, dtype=torch.float), labels.to(device,  dtype=torch.int64)
    labels_ = classifier.forward(data)
    labels_ = torch.argmax(labels_, dim=1)
    test_acc+=(sum(labels_==labels)/batchsize)
    fpr +=(sum(labels_!=labels)/batchsize)
    for i in range(batchsize):
        label=labels[i]
        test_acc_matrix[label][labels_[i]]+=1
        test_acc_dict[label][0]+=1
        if labels_[i]==label:
            test_acc_dict[label][1]+=1

test_acc = test_acc/len(testLoader_fake)
fpr = fpr/len(testLoader_fake)
print("test acc:{}, FPR: {}".format(test_acc, fpr))
test_acc_spt = torch.round(test_acc_dict[:,1]/test_acc_dict[:,0], decimals=4)
print("test_acc_spt: ", test_acc_spt)
test_acc_spt_matrix = torch.zeros((nSDR)) 
for i in range(nSDR):
    sumi = sum(test_acc_matrix[i])
    print("sumi:{}".format(sumi))
    for j in range(nSDR):
        test_acc_matrix[i][j] = test_acc_matrix[i][j]/sumi
    test_acc_spt_matrix[i] = test_acc_matrix[i][i]
    if sum(test_acc_matrix[i])!=1:
        print("Wrong!", sum(test_acc_matrix[i]))
print("test_acc_spt_matix: ", test_acc_spt_matrix)    


idx:0, RFF:0, snr:35, tran_acc:0.5, val_acc:1.0
data shape: (500, 2, 800)


C:\Users\juntingd\AppData\Local\Temp\ipykernel_12180\2205379270.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  datasets_total_fake=torch.tensor(datasets_total_fake, dtype=float).to(device)
C:\Users\juntingd\AppData\Local\Temp\ipykernel_12180\2205379270.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets_total_fake=torch.tensor(targets_total_fake, dtype=torch.int64).to(device)



idx:1, RFF:1, snr:35, tran_acc:0.5938, val_acc:1.0
data shape: (500, 2, 800)

idx:2, RFF:4, snr:35, tran_acc:0.0781, val_acc:1.0
data shape: (500, 2, 800)

idx:3, RFF:34, snr:35, tran_acc:0.9883, val_acc:1.0
data shape: (500, 2, 800)

idx:4, RFF:45, snr:35, tran_acc:0.7656, val_acc:0.9375
data shape: (500, 2, 800)

idx:5, RFF:46, snr:35, tran_acc:0.5352, val_acc:1.0
data shape: (500, 2, 800)

idx:6, RFF:49, snr:35, tran_acc:0.6328, val_acc:0.9102
data shape: (500, 2, 800)

idx:7, RFF:52, snr:35, tran_acc:0.8906, val_acc:0.9102
data shape: (500, 2, 800)

idx:8, RFF:84, snr:35, tran_acc:0.5, val_acc:1.0
data shape: (500, 2, 800)

idx:9, RFF:108, snr:35, tran_acc:0.5078, val_acc:1.0
data shape: (500, 2, 800)

idx:10, RFF:171, snr:35, tran_acc:0.5, val_acc:1.0
data shape: (500, 2, 800)

idx:11, RFF:202, snr:35, tran_acc:0.2227, val_acc:0.9453
data shape: (500, 2, 800)

idx:12, RFF:205, snr:35, tran_acc:0.3633, val_acc:1.0
data shape: (500, 2, 800)
datasets:  torch.Size([6500, 2, 800])
tar

In [21]:
## test fake signal one time one SDR on binary cnn
device='cuda'

nSDR=2
samples_perSDR = 730
classifier = ClassifierPABinary(nSDR=nSDR)
clf_load=torch.load("../../../RFF-RL/Code/checkpoint/Classifier/binaryPA_epoch20.cnn", map_location=device)
# clf_load=torch.load("./checkpoint/ClassifierSDR/11SDR_normseperate_epoch2000.cnn", map_location=device) 
classifier=clf_load["classifier"]

for snr in [35, 5]:
    print("\n-----------------------snr{}----------------------".format(snr))
    path = "./checkpoint/gan/gan_pa_all/snr"+str(snr)+"/"
    ckps = os.listdir(path)
    idx=-1
    batchsize=128
    for ckp in ckps:
        dataset_fake = torch.zeros((samples_perSDR, 2, 800))
        datasets_total_fake=torch.tensor(datasets_all[:][:][:], dtype=float).to(device)
        targets_total_fake=torch.tensor(torch.ones(targets_all.shape[0]), dtype=torch.int64).to(device)

        if len(ckp.split("_"))<2:
            continue
        idx+=1
        RFFlabel = int(ckp.split("_")[0][3:])
        SNR = int(ckp.split("_")[1][3:])
        train_acc = float(ckp.split("_")[3][8:])
        val_acc = float(ckp.split("_")[4][6:])
        print("\nidx:{}, sdr:{}, snr:{}, tran_acc:{}, val_acc:{}".format(idx, RFFlabel, SNR, train_acc, val_acc))
        ckp_load=torch.load(path+ckp, map_location=device)
        generator = ckp_load["generator"]
        discriminator = ckp_load["discriminator"]
        classifier_load = ckp_load["classifier"]

        data=datasets_all[RFFlabel*730:RFFlabel*730+samples_perSDR]
        print("data shape:", data.shape)
        for sigIdx in range(data.shape[0]):
            data[sigIdx]=-1+2*((data[sigIdx] - data[sigIdx].min())/(data[sigIdx].max() - data[sigIdx].min()))
        data = addNoise2Batch(data, snr=SNR, device=device)
        data = torch.tensor(data, dtype=torch.float, device=device)
    
        realdata_loader = Data.DataLoader(data, batch_size=batchsize, shuffle=True, drop_last=True)
        for batch_idx, (data_batch) in enumerate(realdata_loader): 
            dataset_fake[batch_idx*batchsize:(batch_idx+1)*batchsize],mu, logvar = generator.forward(data_batch)
        if (batch_idx+1)*batchsize<samples_perSDR:
            repeat = samples_perSDR-(batch_idx+1)*batchsize
            dataset_fake[-repeat:]=dataset_fake[:repeat]
        
        datasets_total_fake[RFFlabel*730:RFFlabel*730+samples_perSDR]=dataset_fake
        targets_total_fake[RFFlabel*730:RFFlabel*730+samples_perSDR]=torch.zeros(samples_perSDR)

        print("datasets: ", datasets_total_fake.shape)
        print("targets: ", targets_total_fake.shape)
        print("datasets range: ", datasets_total_fake.max(), datasets_total_fake.min())
        print("targets range: ", targets_total_fake.max(), targets_total_fake.min())
        # targets_total_fake = targets_total_fake.reshape((-1,1))

        testdatasets_fake=Data.TensorDataset(datasets_total_fake, targets_total_fake)
        testLoader_fake = Data.DataLoader(testdatasets_fake, batch_size=batchsize, shuffle=True, drop_last=True)

        test_acc=0
        fpr=0
        test_acc_dict = torch.zeros((nSDR,2)).to(device)
        test_acc_matrix = torch.zeros((nSDR, nSDR))
        for batch_idx, (data, labels) in enumerate(testLoader_fake):
            data, labels = data.to(device, dtype=torch.float), labels.to(device,  dtype=torch.int64)
            labels_ = classifier.forward(data)
            labels_ = torch.argmax(labels_, dim=1)
            test_acc+=(sum(labels_==labels)/batchsize)
            for i in range(batchsize):
                label=labels[i]
                test_acc_matrix[label][labels_[i]]+=1
                test_acc_dict[label][0]+=1
                if labels_[i]==label:
                    test_acc_dict[label][1]+=1

        test_acc = test_acc/len(testLoader_fake)
        print("test acc:{}".format(test_acc))
        test_acc_spt = torch.round(test_acc_dict[:,1]/test_acc_dict[:,0], decimals=4)
        print("test_acc_spt: ", test_acc_spt)
        test_acc_spt_matrix = torch.zeros((nSDR)) 
        print("fake pos:", test_acc_matrix[0])
        for i in range(nSDR):
            sumi = sum(test_acc_matrix[i])
            
            print("sumi:{}".format(sumi))
            for j in range(nSDR):
                test_acc_matrix[i][j] = test_acc_matrix[i][j]/sumi
            test_acc_spt_matrix[i] = test_acc_matrix[i][i]
            if sum(test_acc_matrix[i])!=1:
                print("Wrong!", sum(test_acc_matrix[i]))
        print("test_acc_spt_matix:{}, FPR:{} ".format(test_acc_spt_matrix, 1-test_acc_spt_matrix[0]))    


-----------------------snr35----------------------


C:\Users\juntingd\AppData\Local\Temp\ipykernel_12180\2223385192.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets_total_fake=torch.tensor(torch.ones(targets_all.shape[0]), dtype=torch.int64).to(device)



idx:0, sdr:0, snr:35, tran_acc:0.5, val_acc:1.0
data shape: (730, 2, 800)
datasets:  torch.Size([160600, 2, 800])
targets:  torch.Size([160600])
datasets range:  tensor(1.1316, device='cuda:0', dtype=torch.float64, grad_fn=<MaxBackward1>) tensor(-1.0362, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward1>)
targets range:  tensor(1, device='cuda:0') tensor(0, device='cuda:0')
test acc:0.9954645037651062
test_acc_spt:  tensor([0., 1.], device='cuda:0')
fake pos: tensor([  0., 728.])
sumi:728.0
sumi:159784.0
test_acc_spt_matix:tensor([0., 1.]), FPR:1.0 

idx:1, sdr:1, snr:35, tran_acc:0.5938, val_acc:1.0
data shape: (730, 2, 800)
datasets:  torch.Size([160600, 2, 800])
targets:  torch.Size([160600])
datasets range:  tensor(1.0622, device='cuda:0', dtype=torch.float64, grad_fn=<MaxBackward1>) tensor(-1.0276, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward1>)
targets range:  tensor(1, device='cuda:0') tensor(0, device='cuda:0')
test acc:0.9954582452774048
test_acc_spt: 

In [10]:
## test MC simulation
device='cuda' if torch.cuda.is_available() else 'cpu'
nSDR=2

batchsize=32
classifier = ClassifierPABinary(nSDR=nSDR)
clf_load=torch.load("../../../RFF-RL/Code/checkpoint/Classifier/binaryPA_epoch10.cnn", map_location=device)
# clf_load=torch.load("./checkpoint/ClassifierSDR/11SDR_normseperate_epoch2000.cnn", map_location=device) 
classifier=clf_load["classifier"]


datasets_mc=np.load("../../../RFF-RL/MCsim_for_generative_crosstest/MCSim_cubic_603_2_800.npy")
targets=np.zeros((datasets_mc.shape[0]))


for SNR in [35,30,25,20,15,10,5]:
    datasets = addNoise2Batch(datasets_mc, snr=SNR, device=device)
    datasets, targets = torch.tensor(datasets, dtype=torch.float).to(device), torch.tensor(targets, dtype=torch.int64).to(device)
    testdatasets=Data.TensorDataset(datasets, targets)
    testLoader = Data.DataLoader(testdatasets, batch_size=batchsize, shuffle=True, drop_last=True)

    test_acc=0
    for batch_idx, (data, labels) in enumerate(testLoader):
        data, labels = data.to(device, dtype=torch.float), labels.to(device,  dtype=torch.int64)
        labels_ = classifier.forward(data)
        labels_ = torch.argmax(labels_, dim=1)
        test_acc+=(sum(labels_==labels)/batchsize)

    print("SNR:{}, test acc:{},".format(SNR, test_acc))

SNR:35, test acc:0.0,


/var/folders/_2/q6x8216d4lvb82hk5k20k1lh0000gn/T/ipykernel_49604/1108014849.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  datasets, targets = torch.tensor(datasets, dtype=torch.float).to(device), torch.tensor(targets, dtype=torch.int64).to(device)


SNR:30, test acc:0.0,
SNR:25, test acc:0.0,
SNR:20, test acc:0.0,
SNR:15, test acc:0.0,
SNR:10, test acc:0.0,
SNR:5, test acc:0.0,
